---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [6]:
coupon_rate = 0.05              # coupon rate (decimal notation)
coupons_remaining   = 10        # remaining coupons for the bond
days_to_next_coupon = 60        # days to next coupon payment
clean_price = 110               # clean bond price

# BOND CLASS

In [7]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve


class bond:
    def __init__(self, coupon_rate, coupons_remaining, days_to_next=180):
        self.coupon = 100 * coupon_rate / 2
        self.n = coupons_remaining
        self.days = int((coupons_remaining - 1) * 180 + days_to_next)
        self.days_to_next = days_to_next

    def days_remaining(self, day):
        return self.days - day

    def coupons_remaining(self, day):
        return int(np.ceil((self.days_remaining(day)) / 180))

    def fp(self, day):
        x = 180 - self.days_remaining(day) % 180
        return x / 180 if x < 180 else 0

    def accrued(self, day):
        return self.coupon * self.fp(day)

    def dirty(self, day, yld):
        periods = 1 - self.fp(day) + np.arange(self.coupons_remaining(day))
        pvFactors = 1 / (1 + yld / 2) ** periods
        cashFlows = [self.coupon] * (self.coupons_remaining(day) - 1) + [
            100 + self.coupon
        ]
        return np.sum(pvFactors * cashFlows)

    def clean(self, day, yld):
        return self.dirty(day, yld) - self.accrued(day)

# CALCULATIONS

In [8]:
# Initialize bond class
b = bond(coupon_rate, coupons_remaining, days_to_next_coupon)

# compute yield
coupon = coupon_rate * 100 / 2
partial_period = days_to_next_coupon / 180
accrued = (1 - partial_period) * coupon
dirty_price = clean_price + accrued

# compute yield
cashFlows = [coupon] * (coupons_remaining - 1) + [100 + coupon]
def pvFactors(y):
    return 1 / (1 + y / 2) ** np.arange(
        partial_period, partial_period + coupons_remaining
    )
def f(y):
    return dirty_price - np.sum(cashFlows * pvFactors(y))
yld = fsolve(f, x0=coupon_rate).item()

print(f'Dirty price:\t${dirty_price:.2f}')
print(f'Yield:\t\t{yld:.2%}')

Dirty price:	$111.67
Yield:		2.70%


# FIGURE

In [9]:
import plotly.graph_objects as go
# create data series
ylds = np.arange(0, 2 * coupon_rate + 0.001, 0.001)
prices = [b.clean(0, y) for y in ylds]

string = "clean price = $%{y:.2f} when yield = %{x:,.2%}<extra></extra>"
trace = go.Scatter(
    x=ylds, 
    y=prices, 
    mode="lines", 
    hovertemplate=string, 
    line=dict(color='blue')
)
fig = go.Figure(trace)
fig.update_layout(
    xaxis_tickformat=".0%",
    yaxis_tickformat=",.0f",
    yaxis_tickprefix='$',
    xaxis_title="Yield",
    yaxis_title="Clean Price",
    template="plotly_white",
    showlegend=False   
)
fig.show()

# TABLE

In [10]:
df = pd.DataFrame(
    dtype=float, 
    index=range(1, b.n + 1), 
    columns=["cf", "factor", "pv"]
)
df.index.name = "time"
df["cf"] = cashFlows
df["factor"] = 1 / (1 + yld / 2) ** np.arange(1, b.n + 1)
df["pv"]     = df.cf * df.factor
df["factor"] = df.factor.round(3)
df[["cf", "pv"]] = df[["cf", "pv"]].round(2)
df = df.reset_index()
df.columns = ["Time", "Cash Flow", "PV Factor @ Yield", "PV of Cash Flow"]
df

,Time,Cash Flow,PV Factor @ Yield,PV of Cash Flow
0,1,2.5,0.987,2.47
1,2,2.5,0.974,2.43
2,3,2.5,0.961,2.40
3,4,2.5,0.948,2.37
4,5,2.5,0.935,2.34
5,6,2.5,0.923,2.31
6,7,2.5,0.910,2.28
7,8,2.5,0.898,2.25
8,9,2.5,0.886,2.22
9,10,102.5,0.874,89.62
